In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/BeautyJudge/CRNet/model')

In [3]:
!ls *.py

__init__.py  losses.py	models.py


In [4]:
 !cat '/content/drive/My Drive/BeautyJudge/CRNet/model/models.py'

import torch.nn as nn
import torch.nn.functional as F
from torchvision import models


class CRNet(nn.Module):
    """
    definition of CRNet
    """

    def __init__(self):
        super(CRNet, self).__init__()
        self.meta = {'mean': [131.45376586914062, 103.98748016357422, 91.46234893798828],
                     'std': [1, 1, 1],
                     'imageSize': [224, 224, 3]}

        model_ft = models.resnet18(pretrained=True)

        self.model = model_ft
        self.regressor = Regressor(model_ft)
        self.classifier = Classifier(model_ft, num_cls=5)

    def forward(self, x):
        for name, module in self.model.named_children():
            if name != 'fc':
                x = module(x)

        reg_out = self.regressor.forward(x.view(-1, self.num_flat_features(x)))
        cls_out = self.classifier.forward(x.view(-1, self.num_flat_features(x)))

        return reg_out, cls_out

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions 

In [0]:
import sys
sys.path.append('/content/drive/My Drive/T&H/DL Project/CRNet/main')

In [0]:
import models

In [7]:
import os
import sys

import numpy as np
import scipy
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

import cv2
from PIL import Image
from skimage import io
from skimage.transform import resize
import matplotlib.pyplot as plt

sys.path.append('../')
from model.models import CRNet
from config.cfg import cfg


def viz(img_path, model=CRNet()):
    model = model.float()
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model)
    os.chdir('/content/drive/My Drive/BeautyJudge/CRNet/')
    !ls
    model = nn.DataParallel(model)
    model.load_state_dict(torch.load('./model/crnet.pth'))
    model = model.to(device)
    model.eval()

    image = resize(io.imread(img_path), (224, 224), mode='constant')
    image[:, :, 0] -= np.mean(image[:, :, 0])
    image[:, :, 1] -= np.mean(image[:, :, 1])
    image[:, :, 2] -= np.mean(image[:, :, 2])

    image = np.transpose(image, [2, 0, 1])
    input = torch.from_numpy(image).unsqueeze(0).float()
    input = input.to(device)

    for idx, module in model.named_children():
        if idx == 'model':
            for idx, mod in module.named_children():
                # print(idx)
                if idx != 'avgpool':
                    input = mod(input)
                else:
                    # mat = np.transpose(input[0, [1, 1, 1], :, :].data.cpu().numpy(), [1, 2, 0])
                    mat = np.transpose(input[0, :, :, :].data.cpu().numpy(), [1, 2, 0])
                    mat = np.mean(mat, axis=2).reshape([mat.shape[0], mat.shape[0], 1])
                    print(mat.shape)
                    # mat = resize(mat, (224, 224), mode='constant')
                    mat = cv2.resize(mat, (224, 224))
                    org = resize(io.imread(img_path), (224, 224), mode='constant')

                    dst = np.zeros([224, 224, 3])
                    dst[:, :, 0] = 0.2 * org[:, :, 0] + 0.8 * mat
                    dst[:, :, 1] = 0.2 * org[:, :, 1] + 0.8 * mat
                    dst[:, :, 2] = 0.2 * org[:, :, 2] + 0.8 * mat

                    # dst = org + mat

                    # plt.figure("Image")
                    # plt.imshow(org)
                    # plt.axis('on')
                    # plt.title('image')
                    # plt.show()

                    # cv2.imshow('ft', dst)
                    # cv2.waitKey()

                    if not os.path.exists('./feature_viz/'):
                        os.makedirs('./feature_viz/')

                    scipy.misc.imsave('./feature_viz/' + os.path.basename(img_path).split('.')[0] + '.jpg', dst)
                    break


if __name__ == '__main__':
    os.chdir('/content/drive/My Drive/BeautyJudge/CRNet')
    !ls
    filelist = ["feature_viz/"+"SCUT-FBP-%d.jpg" % _ for _ in [101, 57, 242, 380, 192, 469, 241, 174]]
    print(filelist)
    for f in filelist:
        viz(f, CRNet())

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 156MB/s] 


BeautyGanCrop  Crop.zip  deep_feature.png  main       scutfbp.png
config	       cvsplit	 feature_viz	   model      util
Crop_unzip     data	 LICENSE	   README.md
['feature_viz/SCUT-FBP-101.jpg', 'feature_viz/SCUT-FBP-57.jpg', 'feature_viz/SCUT-FBP-242.jpg', 'feature_viz/SCUT-FBP-380.jpg', 'feature_viz/SCUT-FBP-192.jpg', 'feature_viz/SCUT-FBP-469.jpg', 'feature_viz/SCUT-FBP-241.jpg', 'feature_viz/SCUT-FBP-174.jpg']
BeautyGanCrop  Crop.zip  deep_feature.png  main       scutfbp.png
config	       cvsplit	 feature_viz	   model      util
Crop_unzip     data	 LICENSE	   README.md
BeautyGanCrop  Crop.zip  deep_feature.png  main       scutfbp.png
config	       cvsplit	 feature_viz	   model      util
Crop_unzip     data	 LICENSE	   README.md
BeautyGanCrop  Crop.zip  deep_feature.png  main       scutfbp.png
config	       cvsplit	 feature_viz	   model      util
Crop_unzip     data	 LICENSE	   README.md
BeautyGanCrop  Crop.zip  deep_feature.png  main       scutfbp.png
config	       cvsplit	 feature

In [0]:
import os
os.chdir('/content/drive/My Drive/BeautyJudge/CRNet/main')

In [9]:
import os
import sys

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision import models, transforms

sys.path.append('../')
from model.models import CRNet
from model.losses import CRLoss
from data.data_loder import ScutFBPDataset, HotOrNotDataset, BeautyGAN
from util.utils import mkdirs_if_not_exist
from config.cfg import cfg


def train_model(model, train_dataloader, test_dataloader, pred_dataloader,criterion, optimizer, scheduler, num_epochs=25,
                inference=False):
    model = model.float()
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model)
    model = model.to(device)

    if not inference:
        print('Start training CRNet...')
        os.chdir('/content/drive/My Drive/BeautyJudge/CRNet/Crop_unzip/Crop')
        for epoch in range(num_epochs):
            model.train()
            scheduler.step()

            running_loss = 0.0
            for i, data in enumerate(train_dataloader, 0):
                inputs, scores, classes = data['image'], data['score'], data['class']

                inputs = inputs.to(device)
                scores = scores.to(device)
                classes = classes.to(device)

                optimizer.zero_grad()

                inputs = inputs.float()
                scores = scores.float().view(cfg['batch_size'], 1)
                # classes = classes.int().view(cfg['batch_size'], 3)

                reg_out, cls_out = model(inputs)
                loss = criterion(cls_out, classes, reg_out, scores)
                loss.backward()
                optimizer.step()

                # print statistics
                running_loss += loss.item()
                if i % 10 == 9:  # print every 10 mini-batches
                    print('[%d, %5d] loss: %.3f' %
                          (epoch + 1, i + 1, running_loss / 10))
                    running_loss = 0.0

        print('Finished training CRNet...\n')
        print('Saving trained model...')
        model_path_dir = './model'
        mkdirs_if_not_exist(model_path_dir)
        torch.save(model.state_dict(), os.path.join(model_path_dir, 'crnet.pth'))
        print('CRNet has been saved successfully~')

    else:
        print('Loading pre-trained model...')
        model.load_state_dict(torch.load(os.path.join('./model/crnet.pth')))

    model.eval()

    print('Start testing CRNet...')
    predicted_labels = []
    gt_labels = []
    filenames = []
    for data in test_dataloader:
        images, scores, classes, filename = data['image'], data['score'], data['class'], data['filename']
        images = images.to(device)

        reg_out, cls_out = model.forward(images)

        # bat_list = []
        # for out in F.softmax(cls_out).to("cpu"):
        #     tmp = 0
        #     for i in range(0, 3, 1):
        #         tmp += out[i] * (i - 1)
        #     bat_list.append(float(tmp.detach().numpy()))

        # predicted_labels += (0.6 * reg_out.to("cpu").detach().numpy() + 0.4 * np.array(bat_list)).tolist()

        predicted_labels += reg_out.to("cpu").detach().numpy().tolist()
        gt_labels += scores.to("cpu").detach().numpy().tolist()
        filenames += filename

    print('Start predicting data...')
    preds = []
    pred_filenames=[]
    for data in pred_dataloader:
        images_pred, filename_pred = data['image'], data['filename']
        images_pred = images_pred.to(device)

        reg_out, cls_out = model.forward(images_pred)

        preds += reg_out.to("cpu").detach().numpy().tolist()
        pred_filenames += filename_pred

    from sklearn.metrics import mean_absolute_error, mean_squared_error

    mae_lr = round(mean_absolute_error(np.array(gt_labels), np.array(predicted_labels).ravel()), 4)
    rmse_lr = round(np.math.sqrt(mean_squared_error(np.array(gt_labels), np.array(predicted_labels).ravel())), 4)
    pc = round(np.corrcoef(np.array(gt_labels), np.array(predicted_labels).ravel())[0, 1], 4)

    print('===============The Mean Absolute Error of CRNet is {0}===================='.format(mae_lr))
    print('===============The Root Mean Square Error of CRNet is {0}===================='.format(rmse_lr))
    print('===============The Pearson Correlation of CRNet is {0}===================='.format(pc))

    col = ['filename', 'gt', 'pred']
    df = pd.DataFrame([[filenames[i], gt_labels[i], predicted_labels[i][0]] for i in range(len(gt_labels))],
                      columns=col)
    df.to_excel("./output.xlsx", sheet_name='Output', index=False)
    print('Output Excel has been generated~')

    cols_pred = ['filename', 'pred']
    df = pd.DataFrame([[pred_filenames[i], preds[i]] for i in range(len(preds))],
                      columns=cols_pred)
    df.to_excel("./output_beautygan.xlsx", sheet_name='Output', index=False)
    print('Pred Output Excel has been generated~')


def run_crnet_scutfbp(model, epoch=30):
    criterion = CRLoss()

    optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)

    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=20, gamma=0.1)
    os.chdir('/content/drive/My Drive/BeautyJudge/CRNet/')
    !ls
    df = pd.read_excel('cvsplit/SCUT-FBP.xlsx', sheet_name='Sheet1')
    X_train, X_test, y_train, y_test = train_test_split(df['Image'].tolist(), df['Attractiveness label'],
                                                        test_size=0.2, random_state=0)

    print('start loading SCUT-FBP dataset...')
    train_dataset = ScutFBPDataset(f_list=X_train, f_labels=y_train, transform=transforms.Compose([
        transforms.Resize(224),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[1, 1, 1])
    ]))

    test_dataset = ScutFBPDataset(f_list=X_test, f_labels=y_test, transform=transforms.Compose([
        transforms.Resize(224),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[1, 1, 1])
    ]))

    pred_list=[x for x in range(1126)]

    pred_dataset = BeautyGAN(f_list=pred_list, transform=transforms.Compose([
        transforms.Resize(224),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[1, 1, 1])
    ]))

    train_dataloader = DataLoader(train_dataset, batch_size=cfg['batch_size'],
                                  shuffle=True, num_workers=4)
    test_dataloader = DataLoader(test_dataset, batch_size=cfg['batch_size'],
                                 shuffle=False, num_workers=4)
    
    pred_dataloader = DataLoader(pred_dataset, batch_size=cfg['batch_size'],
                                 shuffle=False, num_workers=4)

    print('finish loading SCUT-FBP dataset...')
    train_model(model=model, train_dataloader=train_dataloader, test_dataloader=test_dataloader, 
                pred_dataloader=pred_dataloader,criterion=criterion, optimizer=optimizer_ft, scheduler=exp_lr_scheduler, num_epochs=epoch,
                inference=False)


def run_crnet_eccv(model, cv_split=1, epoch=30):
    """
    train and test ECCV HotOrNot dataset
    :param cv_split:
    :return:
    """
    criterion = CRLoss()

    optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)

    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=20, gamma=0.1)

    print('start loading ECCV HotOrNot dataset...')
    train_dataset = HotOrNotDataset(cv_split=cv_split, train=True, transform=transforms.Compose([
        transforms.Resize(227),
        transforms.RandomCrop(224),
        transforms.RandomRotation(30),
        transforms.ColorJitter(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[1, 1, 1])
    ]))

    test_dataset = HotOrNotDataset(cv_split=cv_split, train=False, transform=transforms.Compose([
        transforms.Resize(227),
        transforms.RandomCrop(224),
        transforms.RandomRotation(30),
        transforms.ColorJitter(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5,0.5],
                             std=[1, 1, 1,1])
    ]))

    train_dataloader = DataLoader(train_dataset, batch_size=cfg['batch_size'],
                                  shuffle=True, num_workers=4, drop_last=True)
    test_dataloader = DataLoader(test_dataset, batch_size=cfg['batch_size'],
                                 shuffle=False, num_workers=4, drop_last=True)

    print('finish loading ECCV HotOrNot dataset...')

    train_model(model=model, train_dataloader=train_dataloader, test_dataloader=test_dataloader,
                criterion=criterion, optimizer=optimizer_ft, scheduler=exp_lr_scheduler, num_epochs=epoch,
                inference=False)


if __name__ == '__main__':
    run_crnet_scutfbp(model=CRNet(), epoch=30)
    # run_crnet_eccv(model=CRNet(), cv_split=1, epoch=30)


BeautyGanCrop  Crop.zip  deep_feature.png  main       scutfbp.png
config	       cvsplit	 feature_viz	   model      util
Crop_unzip     data	 LICENSE	   README.md
start loading SCUT-FBP dataset...
finish loading SCUT-FBP dataset...
Start training CRNet...


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1,    10] loss: 4.165
[1,    20] loss: 1.719
[2,    10] loss: 1.367
[2,    20] loss: 1.249
[3,    10] loss: 1.033
[3,    20] loss: 1.092
[4,    10] loss: 0.839
[4,    20] loss: 0.986
[5,    10] loss: 0.910
[5,    20] loss: 0.753
[6,    10] loss: 0.745
[6,    20] loss: 0.734
[7,    10] loss: 0.774
[7,    20] loss: 0.785
[8,    10] loss: 0.773
[8,    20] loss: 0.754
[9,    10] loss: 0.690
[9,    20] loss: 0.679
[10,    10] loss: 0.625
[10,    20] loss: 0.637
[11,    10] loss: 0.667
[11,    20] loss: 0.582
[12,    10] loss: 0.641
[12,    20] loss: 0.562
[13,    10] loss: 0.604
[13,    20] loss: 0.585
[14,    10] loss: 0.496
[14,    20] loss: 0.613
[15,    10] loss: 0.574
[15,    20] loss: 0.573
[16,    10] loss: 0.552
[16,    20] loss: 0.551
[17,    10] loss: 0.547
[17,    20] loss: 0.521
[18,    10] loss: 0.515
[18,    20] loss: 0.525
[19,    10] loss: 0.470
[19,    20] loss: 0.509
[20,    10] loss: 0.501
[20,    20] loss: 0.456
[21,    10] loss: 0.434
[21,    20] loss: 0.481
[22,    10

In [10]:
print(os.path)


<module 'posixpath' from '/usr/lib/python3.6/posixpath.py'>
